In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import wandb
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# ==========================================
# 1. 설정 (Configuration)
# ==========================================
DATA_PATH = "/content/drive/MyDrive/nlp_project2/train_dataset_final.jsonl"
MODEL_NAME = "KETI-AIR/ke-t5-base"
OUTPUT_DIR = "/content/drive/MyDrive/nlp_project2/ke-t5_checkpoints" # 체크포인트 저장 경로
FINAL_SAVE_DIR = "/content/drive/MyDrive/nlp_project2/final_ke-t5_model" # 최종 모델 저장 경로

# 입력/출력 최대 길이 설정 (데이터 길이에 따라 조절 가능)
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

# ==========================================
# 2. 데이터셋 로드 및 전처리
# ==========================================
# JSONL 파일 로드
dataset = load_dataset("json", data_files=DATA_PATH)

# 학습/검증 데이터 분리 (예: 9:1), 데이터가 적다면 분리하지 않고 전체를 학습에 쓸 수도 있습니다.
# 여기서는 전체 데이터를 학습용으로 쓴다고 가정하거나, 필요시 아래 주석을 해제하세요.
dataset = dataset['train'].train_test_split(test_size=0.1)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    inputs = examples["input"]
    targets = examples["target"]

    # 입력 토크나이징
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        padding="max_length",
        truncation=True
    )

    # 출력(타겟) 토크나이징
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_TARGET_LENGTH,
            padding="max_length",
            truncation=True
        )

    # 패딩 토큰의 ID를 -100으로 치환하여 손실(Loss) 계산에서 제외
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 데이터셋에 전처리 적용
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# ==========================================
# 3. 모델 불러오기
# ==========================================
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# ==========================================
# 4. 학습 인자 설정 (A100 최적화)
# ==========================================
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="steps",     # 10000 steps마다 평가
    eval_steps=10000,
    save_strategy="steps",            # 10000 steps마다 체크포인트 저장
    save_steps=10000,
    save_total_limit=3,               # 최근 3개의 체크포인트만 유지 (용량 절약)
    learning_rate=2e-4,               # T5는 보통 1e-4 ~ 3e-4 정도 사용
    per_device_train_batch_size=32,   # A100 메모리가 크므로 32~64 가능 (OOM 발생 시 줄이세요)
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    num_train_epochs=3,              # 학습 에폭 수 (데이터 양에 따라 조절)
    predict_with_generate=True,       # 평가 시 생성 모드 사용
    fp16=False,                       # A100에서는 bf16이 더 좋음
    bf16=True,                        # A100 전용 설정 (Brain Float 16)
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=500,
    report_to="wandb",
    load_best_model_at_end=True,      # 학습 종료 후 가장 성능 좋은 모델 로드
    dataloader_num_workers=4,
)

# Data Collator 설정
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ==========================================
# 5. Trainer 초기화 및 학습 시작
# ==========================================
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

print("학습을 시작합니다...")
trainer.train()

# ==========================================
# 6. 최종 모델 저장
# ==========================================
print(f"학습 완료! 모델을 {FINAL_SAVE_DIR}에 저장합니다.")
trainer.save_model(FINAL_SAVE_DIR)
tokenizer.save_pretrained(FINAL_SAVE_DIR)

print("모든 작업이 완료되었습니다.")

In [ ]:
from transformers import pipeline

# 저장된 모델 불러오기
final_model_path = "/content/drive/MyDrive/nlp_project2/final_ke-t5_model"
generator = pipeline("text2text-generation", model=final_model_path, tokenizer=final_model_path, device=0)

# 테스트 입력 (데이터셋 포맷에 맞춤)
test_input = "translate to sql:\nquestion <3층메디칼약국의 주소를 알려줘>\nschema <table TB_PHARMACY_OPERATE_INFO columns DUTYADDR(주소), DUTYNAME(약국명), HPID(약국아이디), DUTYTEL1(대표전화), DUTYTIME1C(월요일 진료 마감 시간), DUTYTIME2C(화요일 진료 마감 시간), DUTYTIME3C(수요일 진료 마감 시간), DUTYTIME4C(목요일 진료 마감 시간)>"

# 예측 생성
result = generator(test_input, max_length=128)
print("Input:", test_input)
print("Generated SQL:", result[0]['generated_text'])
print("정답 SQL: SELECT DUTYADDR FROM TB_PHARMACY_OPERATE_INFO WHERE DUTYNAME = '3층메디칼약국'")